<!--NAVIGATION-->
< [Preparation](Preparation.ipynb) | [Contents](Contents.ipynb) | [Condition 2](Condition_2.ipynb) >

# Condition 1: Wells within 150 Meters of Houses or Roads
## Introduction

Now that we've converted the GeoTIFF files to PCRaster format and have checked their properties, it's time to proceed with analysing the three conditions:

* Condition 1: Wells within 150 meters of houses or roads
* Condition 2: No industry, mine, or landfill within 300 meters from wells
* Condition 3: Wells less than 40 meters deep

For Condition 1 we'll first look at the houses and we'll do the following:

1. Create a boolean layer with True for houses and False for other buildings
2. Create zones of 150 meters around the houses

Then we'll repeat the steps for roads.

## Create a Boolean Layer with True for Houses and False for Other Buildings
We usually type the commands at the Python prompt. In this Jupyter Notebook we simulate this. I.e. everything you type in the fields is equal to a command at the Python prompt.

Let's first load the `pcraster` module.

In [ ]:
from pcraster import *

Remember that our data is stored in the Data folder, while our Python is running in the parent folder. For convenience it's useful to change to the Data folder so all our input and output files are read and written there respectively.

With the `os` library we can use command line command.

In [ ]:
import os

Let's check the current working directory.

In [ ]:
print(os.getcwd())

We can use an equivalent of the `cd` command to change the directory: `os.chdir(stringWithPath)`, where `stringWithPath` defines the path.

In [ ]:
os.chdir(".\Data")

Check where we are now. Make sure that we are in the Data folder of this tutorial.

In [ ]:
print(os.getcwd())

Now we're ready to read `buildg.map` from disk and use it for map algebra with PCRaster.
We can read maps using `readmap(stringWithPathToFile)`.

In [ ]:
Buildings = readmap("buildg.map")

Now we have read `buildg.map` from disk and can refer to it with the variable `Buildings`.
There are different ways to visualise the map:
* In Python we can use the `aguila` operator: `aguila(Buildings)`
* At the command prompt we can use the `Aguila` command with the file name: `aguila buildg.map`
* Use Python plotting tools

We're going to use the last option in this Jupyter notebook. But you can try the other ones too if you run this notebook locally.

In order to plot the raster with Matplotlib we need to import the libraries:
```
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
```

Then we define a function that need the PCRaster map as input.
In the function we convert the PCRaster map to a 2D NumPy array with `pcr2numpy`. The function also needs a value for the missing values. Here we'll use -9999.

Because different raster data types need to be styled in a different way we'll define different colour maps. With `dataType()` we can get the PCRaster datatype (`VALUESCALE`). You can find different colour maps [here](https://matplotlib.org/examples/color/colormaps_reference.html). For nominal rasters here we use `'Set1'` and for scalar rasters we use `'cool'`. For boolean rasters we define a `ListedColormap` with red for False and green for True. There's much more that we can tweak to get a good legend, but that's beyond the scope of this tutorial.

With `plt.axis('off')` we don't display the axes, because it is a map.

In [ ]:
%matplotlib inline
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt

def showmap(pcrastermap):
    MapArray = pcr2numpy(pcrastermap,-9999)
    if pcrastermap.dataType() == VALUESCALE.Nominal:
        cmap = 'Set1'
        plt.imshow(MapArray,cmap=cmap)
        plt.colorbar()
        
    elif pcrastermap.dataType() == VALUESCALE.Boolean:
        cmap = ListedColormap(["red","green"])
        plt.imshow(MapArray,cmap=cmap)
        plt.colorbar(ticks=[0,1])
    else:
        cmap = 'cool'
        plt.imshow(MapArray,cmap=cmap)
        plt.colorbar()
    
    plt.axis('off')
    plt.show()

Now let's visualise `Buildings`.

In [ ]:
showmap(Buildings)

Raster data has no attribute table. The values in the `Buildings` raster represent the following classes:

| Value | Class |
|:-------:|:-------:|
| 0 | None |
| 1 | House |
| 2 | Public building |
| 3 | Landfill |
| 4 | Industry |
| 5 | Mine |

We first need to create a boolean layer with True for houses and False for other buildings.

In [ ]:
Houses = Buildings == 1

Here we have used the `==` operator. This creates a boolean map with value 1 for all cells that are equal to 1 and 0 for all other cells. Here you can find more info about [operators in PCRaster](https://pcraster.geo.uu.nl/pcraster/4.3.0/documentation/python/quickstart.html#operators).
Let's check the result:

In [ ]:
showmap(Houses)

## Create Zones of 150 Meters Around the Houses
The next step is to create zones of 150 m around the houses.

We can use the [`spreadmaxzone` function](https://pcraster.geo.uu.nl/pcraster/4.3.0/documentation/pcraster_manual/sphinx/op_spreadmaxzone.html).

The syntax is `Result = spreadmaxzone(points, initialfrictiondist, friction, max_distance)`
In our case `points` are the `Houses`, `initialfrictiondist = 0`, `friction = 1` and `max_distance = 150`. `intialfrictiondistance = 0` means that their is no friction at the location of the houses an with `friction = 1` we calculate the distance from other pixels in an equal way without different weights.

In [ ]:
houses150m = spreadmaxzone(Houses, 0, 1, 150)

Visualise the result.

In [ ]:
showmap(houses150m)

We can save this result to disk using the `report` function. The first argument is the variable name of the map that you want to write to disk. The second argument is the string of the file name.

In [ ]:
report(houses150m,"houses150m.map")

Verify that `houses150m.map` is stored in the Data folder and visualise the result by using aguila from the command prompt:
```
aguila houses150.map
```

## Create Zones of 150 Meters Around the Roads
In a similar way we can now calculate the 150 m buffer around the roads.

The values in `roads.map` represent the following classes:

| Value | Class |
|:-------:|:-------:|
| 0 | No road |
| 1 | Dirt road |
| 2 | Tarmac |

Do the same steps as for `houses150m` and save the result to `roads150m.map`.

In [ ]:
Roads = readmap("roads.map")

In [ ]:
IsRoad = Roads != 0

In [ ]:
showmap(IsRoad)

In [ ]:
roads150m = spreadmaxzone(IsRoad, 0, 1, 150)

Visualise the result.

In [ ]:
showmap(roads150m)

Save the result to disk with the name `roads150m.map` and visualise using aguila at the command prompt to check the results.

<!--NAVIGATION-->
< [Preparation](Preparation.ipynb) | [Contents](Contents.ipynb) | [Condition 2](Condition_2.ipynb) >